# Demo of functionality available in wildcat-api-python

This demo shows the available functionality using default settings for parameters. For more detail on what you can configure as a user, see the documentation and description of individual methods in the WildcatApi-class.

### Before you start

To be able to run this notebook, you should:
- install the wildcat-api-python package in a virtual environment (`pip install -e .` from the main directory of the repository).
- install the requirements in requirements.txt (if not already installed automatically in the previous step).
- create a file '.env' in the root of the wildcat-api-python-repository, containing your Cluey credentials. These will be read in this notebook to log in. The file should look like this:
```
# Cluey credentials
USERNAME=your_username
PASSWORD=your_password
```

## Configuration

In [1]:
from wildcatpy.api_calls import WildcatApi
from wildcatpy.src import helper_functions as helpers
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import json
import os
import pandas as pd

In [2]:
plt.style.use('ggplot')

In [3]:
load_dotenv()

True

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
username = os.getenv("USERNAME") # you can also type your password here manually
password = os.getenv('PASSWORD') # You can also type your username here manually

## Demo-time, here we go!

In [6]:
api_call = WildcatApi(username, password)

### Login 

In [7]:
# expected output if successful: '<Response [200]>'
api_call.login(username, password)

<Response [200]>

In [8]:
# It is not necessary to log out, but you can do so by calling:
# api_call.logout()

### Obtain the groups you have access to

In [9]:
info = api_call.get_groups()

In [10]:
info.head()

,name,description,n_records
0,focus-project-2435800,3june2020,253
1,focus-project-7136973,Cluey Data Collector,543
2,focus-project-7811010,Creekish mountains,124


In [11]:
# for other functionality, you can specify a group to extract data from
groups = "focus-project-7136973"

### Get observations

Note that you can control the scope (e.g. coordinates) of these observations in more detail than done here.

TODO: provide detailed instructions.

In [12]:
observations = api_call.observation_extractor(groups=groups, operator=["intersects"])

In [13]:
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855 entries, 0 to 854
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   entityId         855 non-null    object
 1   entityType       855 non-null    object
 2   entityName       855 non-null    object
 3   projectId        855 non-null    object
 4   projectName      855 non-null    object
 5   observationType  855 non-null    object
 6   when             855 non-null    object
 7   where            855 non-null    object
 8   agentName        855 non-null    object
 9   conceptLabel     825 non-null    object
 10  conceptId        832 non-null    object
dtypes: object(11)
memory usage: 73.6+ KB


In [14]:
observations['agentName'] = '#####'
observations.head()

,entityId,entityType,entityName,projectId,projectName,observationType,when,where,agentName,conceptLabel,conceptId
0,O7136973-n3f836bf15b0b5df5,infrastructure,Observation,7136973,Cluey Data Collector,infrastructure,2023-03-09T16:55:40+0200,"{'coordinates': [30.8177338, -24.2150175], 'ty...",#####,Point of Interest,https://sensingclues.poolparty.biz/SCCSSOntolo...
1,O7136973-n3f836bf15b0b5df5,infrastructure,Observation,7136973,Cluey Data Collector,infrastructure,2023-03-09T16:55:40+0200,"{'coordinates': [30.8177338, -24.2150175], 'ty...",#####,Chili-fence,https://sensingclues.poolparty.biz/SCCSSOntolo...
2,O7136973-nc754e01c51f0962,offence,Observation,7136973,Cluey Data Collector,offence,2023-03-09T09:53:23+0200,"{'coordinates': [30.8179859141918, -24.2148380...",#####,Illegal,https://sensingclues.poolparty.biz/SCCSSOntolo...
3,O7136973-nc754e01c51f0962,offence,Observation,7136973,Cluey Data Collector,offence,2023-03-09T09:53:23+0200,"{'coordinates': [30.8179859141918, -24.2148380...",#####,Snare,https://sensingclues.poolparty.biz/SCCSSOntolo...
4,O7136973-nc754e01c51f0962,offence,Observation,7136973,Cluey Data Collector,offence,2023-03-09T09:53:23+0200,"{'coordinates': [30.8179859141918, -24.2148380...",#####,De-snared,https://sensingclues.poolparty.biz/SCCSSOntolo...


### Get track metadata

Note that you can control the scope (e.g. coordinates) of these observations in more detail than done here.

TODO: provide detailed instructions.

In [15]:
tracks = api_call.track_extractor(groups=groups, time_until="23:59:54-00:00")

In [16]:
tracks['agentName'] = '#####'
tracks.head()

,entityId,entityType,projectId,projectName,featureType,length,startWhen,endWhen,agentName
0,G7136973-58131b11-c62e-45e6-86f8-4dc58dca12d7,track,7136973,Cluey Data Collector,track/onSnareSweep,0.025799,2023-03-10T06:48:06+0200,2023-03-10T06:48:17+0200,#####
1,G7136973-d82a6180-3060-4f45-8fec-c51118f18703,track,7136973,Cluey Data Collector,track/onDuty,1.132286,2023-02-23T13:24:39+0200,2023-02-23T13:45:36+0200,#####
2,G7136973-916c161e-5b60-45d7-a1a9-d323b6cc80d2,track,7136973,Cluey Data Collector,track/onCensus,0.065548,2023-02-22T13:01:52+0200,2023-02-22T13:02:34+0200,#####
3,G7136973-2e1a50ef-34cc-430a-b1be-f29d373210de,track,7136973,Cluey Data Collector,track/onCensus,0.077187,2023-02-21T15:54:30+0200,2023-02-21T19:19:36+0200,#####
4,G7136973-1cd7b14a-09fd-4531-875c-08baeeeed2a2,track,7136973,Cluey Data Collector,track/onSafari,0.134567,2023-02-03T00:00:17+0200,2023-02-03T05:47:42+0200,#####


### Add geosjon to track

Note that you can control the scope (e.g. coordinates) of these observations in more detail than done here.

TODO: provide detailed instructions.

In [17]:
tracks_geo = api_call.add_geojson_to_track(tracks)

In [18]:
tracks_geo['agentName'] = '#####'
tracks_geo.head()

,entityId,entityType,projectId,projectName,featureType,length,startWhen,endWhen,agentName,patrolDuration,...,EntityId,EntityClass,EntityType,AgentRef,GroupRef,Name,Description,DateTimes,Tags,geometry
0,G7136973-58131b11-c62e-45e6-86f8-4dc58dca12d7,track,7136973,Cluey Data Collector,track/onSnareSweep,0.026,2023-03-10 06:48:06+02:00,2023-03-10 06:48:17+02:00,#####,0.003,...,G7136973-58131b11-c62e-45e6-86f8-4dc58dca12d7,NaN,track/onSnareSweep,A7131207983971604413,7136973,58131b11-c62e-45e6-86f8-4dc58dca12d7,"onSnareSweep, whatever , 5",2023-03-10 06:48:17+02:00,onSnareSweep,None
1,G7136973-d82a6180-3060-4f45-8fec-c51118f18703,track,7136973,Cluey Data Collector,track/onDuty,1.132,2023-02-23 13:24:39+02:00,2023-02-23 13:45:36+02:00,#####,0.349,...,G7136973-d82a6180-3060-4f45-8fec-c51118f18703,NaN,track/onDuty,A952187958042535052,7136973,d82a6180-3060-4f45-8fec-c51118f18703,"onDuty, 3",2023-02-23 13:45:36+02:00,onDuty,None
2,G7136973-916c161e-5b60-45d7-a1a9-d323b6cc80d2,track,7136973,Cluey Data Collector,track/onCensus,0.066,2023-02-22 13:01:52+02:00,2023-02-22 13:02:34+02:00,#####,0.012,...,G7136973-916c161e-5b60-45d7-a1a9-d323b6cc80d2,NaN,track/onCensus,A7131207983971604413,7136973,916c161e-5b60-45d7-a1a9-d323b6cc80d2,"onCensus, CSI team case zero, 3",2023-02-22 13:02:34+02:00,onCensus,None
3,G7136973-2e1a50ef-34cc-430a-b1be-f29d373210de,track,7136973,Cluey Data Collector,track/onCensus,0.077,2023-02-21 15:54:30+02:00,2023-02-21 19:19:36+02:00,#####,3.418,...,G7136973-2e1a50ef-34cc-430a-b1be-f29d373210de,NaN,track/onCensus,A7131207983971604413,7136973,2e1a50ef-34cc-430a-b1be-f29d373210de,"onCensus, CSI team case zero, 3",2023-02-21 19:19:36+02:00,onCensus,None
4,G7136973-1cd7b14a-09fd-4531-875c-08baeeeed2a2,track,7136973,Cluey Data Collector,track/onSafari,0.135,2023-02-03 00:00:17+02:00,2023-02-03 05:47:42+02:00,#####,5.790,...,G7136973-1cd7b14a-09fd-4531-875c-08baeeeed2a2,NaN,track/onSafari,A7131207983971604413,7136973,1cd7b14a-09fd-4531-875c-08baeeeed2a2,"onSafari, morning , 6",2023-02-03 05:47:42+02:00,onSafari,None


### Get all available layers (projects)

In [19]:
layers = api_call.get_all_layers()

In [20]:
layers

,pid,lid,layerName,description,geometryType


### Get details (features) for an individual layer

In [21]:
df = api_call.layer_feature_extractor(project_name='test_polygon')

#### [optional] Plot available geometries (requires Folium)

In [22]:
# !pip install folium

In [23]:
import folium

In [ ]:
poly_map = folium.Map([51.9244, 4.4777], zoom_start=8, tiles='cartodbpositron')
for _, geometry in df['geometry'].items():
    folium.GeoJson(geometry).add_to(poly_map)
folium.LatLngPopup().add_to(poly_map)
poly_map

### Get all available concepts and their hierarchy

As shown later in this notebook, you can use this information to subsequently query:
- the details for a specific concept
- check the occurrence of each concept in the group(s) of observations you have access to.

In [25]:
hierarchy = api_call.get_hierarchy()

In [26]:
hierarchy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            885 non-null    object
 1   parent        885 non-null    object
 2   label         885 non-null    object
 3   altLabels     566 non-null    object
 4   children      128 non-null    object
 5   isTopConcept  885 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 35.6+ KB


### Get details for specific concepts in the hierarchy

You can get information on children or the parents of a concept in the hierarchy by filtering on its label or id. Use the available helper functions to do so. For example, you could do the following for the concept of a "Kite" (oid = "https://sensingclues.poolparty.biz/SCCSSOntology/222"):

```
oid = "https://sensingclues.poolparty.biz/SCCSSOntology/222"
helpers.get_children_for_id(hierarchy, oid)
helpers.get_parent_for_id(hierarchy, oid)
helpers.get_label_for_id(hierarchy, oid)
```

or, if filtering on the label itself:

```
label = 'Kite'
helpers.get_children_for_label(hierarchy, label)
helpers.get_parent_for_label(hierarchy, label)
helpers.get_id_for_label(hierarchy, label)
```

N.B. Alternatively, you could directly filter the `hierarchy`-dataframe yourself of course.

#### Tell me, what animal belongs to this concept id?

In [27]:
oid = "https://sensingclues.poolparty.biz/SCCSSOntology/222"
helpers.get_label_for_id(hierarchy, oid)

'Kite'

#### Does this Kite have any children?

In [28]:
label = 'Kite'
children_label = helpers.get_children_for_label(hierarchy, label)
children_label

['https://sensingclues.poolparty.biz/SCCSSOntology/223',
 'https://sensingclues.poolparty.biz/SCCSSOntology/224']

#### What are the details for these children?

In [29]:
hierarchy.loc[hierarchy['id'].isin(children_label)]

,id,parent,label,altLabels,children,isTopConcept
372,https://sensingclues.poolparty.biz/SCCSSOntolo...,https://sensingclues.poolparty.biz/SCCSSOntolo...,Kite red,"[Milvus milvus, Kite_red, red_kite, red_kite, ...",NaN,False
508,https://sensingclues.poolparty.biz/SCCSSOntolo...,https://sensingclues.poolparty.biz/SCCSSOntolo...,Kite black,"[Milvus migrans, Kite_black, Black kite, Black...",NaN,False


### Count concepts related to observations

Get the number of observations per concept in the ontology (hierarchy), e.g. the number of observations listed as a "Kite" ("https://sensingclues.poolparty.biz/SCCSSOntology/222").

You can filter on for instance:
- `date_from` and `date_until`. 
- A list of child concepts, e.g. by extracting children for the label "Animal sighting" from hierarchy (see example below).
- Note that specifying a range of coordinates (via `coord`) currently does **not** work as a filter.

In [30]:
date_from = '2022-01-01'
date_until = '2023-01-01'
label = 'Animal sighting'
children_label = helpers.get_children_for_label(hierarchy, label)
concept_counts = api_call.get_concept_counts(groups, 
                                             date_from=date_from, date_until=date_until,
                                             concepts=children_label)
concept_counts

,frequency,_value
0,76,https://sensingclues.poolparty.biz/SCCSSOntolo...
1,3,https://sensingclues.poolparty.biz/SCCSSOntolo...
2,7,https://sensingclues.poolparty.biz/SCCSSOntolo...
3,5,https://sensingclues.poolparty.biz/SCCSSOntolo...
4,1,https://sensingclues.poolparty.biz/SCCSSOntolo...
...,...,...
75,1,https://sensingclues.poolparty.biz/SCCSSOntolo...
76,3,https://sensingclues.poolparty.biz/SCCSSOntolo...
77,1,https://sensingclues.poolparty.biz/SCCSSOntolo...
78,76,https://sensingclues.poolparty.biz/SCCSSOntolo...


#### Example: visualize concept counts

To make the visualization intelligible, you can add information on labels from the `hierarchy`-dataframe.

In [ ]:
min_freq = 5
concept_freq = concept_counts.merge(hierarchy, left_on='_value', right_on='id', how='left')
concept_freq['label'] = concept_freq['label'].fillna(concept_freq['_value'])
concept_freq = concept_freq.set_index('label')['frequency'].sort_values(ascending=True)
concept_freq.loc[concept_freq >= min_freq].plot(kind='barh');
plt.title(f"Number of observations per concept in group(s)\n'{groups}' for label '{label}'\n"
          f"[{date_from} to {date_until} and minimum frequency {min_freq}]", 
          fontsize=12);
plt.xlabel('Number of observations per concept label');
plt.ylabel('Label of concept');